In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

ModuleNotFoundError: No module named 'boto3'

### Evaluation
Before doing anything else, we want to create a method to evaluate our models performance. This method should give back us evaluation metrics such as recall, f1, precision, and accuracy. Since we are doing binary prediction I will be using the code used for fraud detection but modifying it a little bit (including f1 score)

In [2]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1 = tp / (tp + 0.5*(fp + fn))
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print("{:<11} {:.3f}".format('F1:', f1))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy}


### Data setup
To set up the sagemaker environment we are going to have to get our sessions, role, bucket, and data. We will also define the helper to delete an endpoint here

In [ ]:
# sagemaker session, role, region
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket name
bucket = sagemaker_session.default_bucket()

In [ ]:
X_test = pd.read_csv("data/X_test")
Y_test = pd.read_csv("data/Y_test")
X_train = pd.read_csv("data/X_train")
Y_train = pd.read_csv("data/Y_train")

In [ ]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

### Linear Learner
We will first be using linear learner, we will try using linear learner raw first and then see how we do if we optimize it for recall. While we want a high f1 score, as that's our primary metric. It is probably more valuable for starbucks to get all potential customers and miss a few, than send promotions to those who would not use them. 

In [ ]:
# import LinearLearner
from sagemaker import LinearLearner

# specify an output path
prefix = 'promotion_success'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner, no recall optimization
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15)

In [ ]:
# convert features/labels to numpy
X_train_np = X_train.astype('float32')
Y_train_np = Y_train.astype('float32')
X_test_np = X_test.astype('float32')
Y_test_np = Y_test.astype('float32')

# create RecordSet
formatted_train_data = linear.record_set(X_train_np, labels=Y_train_np)

In [ ]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

In [ ]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   X_test_np, 
                   Y_test_np, 
                   verbose=True) # verbose means we'll print out the metrics

In [ ]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

In [ ]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.9) # 90% recall

In [ ]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

In [ ]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
print('Metrics for tuned (recall), LinearLearner.\n')

metrics = evaluate(recall_predictor, 
                   X_test_np, 
                   Y_test_np, 
                   verbose=True)

In [ ]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

### Set up XGBoost hyperparmeter tunning


In [ ]:
X_train, X_val, Y_train, Y_val = sklearn.model_selection.train_test_split(X_train, Y_train, test_size=0.33)

In [ ]:
prefix = 'promotion_success/xgboost'
data_dir = 'data/sm'

# X_test.to_csv(os.path.join(data_dir, 'test.csv'), header=False, index=False)

pd.concat([Y_val, X_val], axis=1).to_csv(os.path.join(data_dir, 'validation.csv'), header=False, index=False)
pd.concat([Y_train, X_train], axis=1).to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

test_location = session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
val_location = session.upload_data(os.path.join(data_dir, 'validation.csv'), key_prefix=prefix)
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

In [ ]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

xgb_hyperparameter_tuner = HyperparameterTuner(estimator = xgb,
                                               objective_metric_name = 'validation:f1', 
                                               objective_type = 'Maximize', 
                                               max_jobs = 20, 
                                               max_parallel_jobs = 3,
                                               hyperparameter_ranges = {
                                                    'max_depth': IntegerParameter(3, 8),
                                                    'eta'      : ContinuousParameter(0.05, 0.5),
                                                    'alpha'    : ContinuousParameter(0, 2)
                                                    'min_child_weight': IntegerParameter(1, 10),
                                               })

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data=val_location, content_type='csv')

xgb_hyperparameter_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})
xgb_hyperparameter_tuner.wait()

In [ ]:
xgb_hyperparameter_tuner.best_training_job()

In [ ]:
xgb_attached = sagemaker.estimator.Estimator.attach(xgb_hyperparameter_tuner.best_training_job())

In [ ]:
metrics = evaluate(xgb_attached, 
                   X_test_np, 
                   Y_test_np, 
                   verbose=True)

In [ ]:
delete_endpoint(xgb_attached)